In [ ]:
import sys
sys.path.append("../../")
import csv
import matplotlib.pyplot as plt
import numpy as np
import focusadd
from focusadd.surface.Surface import Surface
from focusadd.coils.CoilSet import CoilSet
from focusadd.lossFunctions.DefaultLoss import default_loss
import numpy as np
import mayavi as maya
from mayavi import mlab
from functools import partial
mlab.init_notebook('x3d',800,800)

In [ ]:
surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0)

In [ ]:
coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
_, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)

# What happens if I increase the surface resolution? Does this change my loss function values?

Let's start with the default surface and get a loss value.

In [ ]:
def draw_coils(r_coils, color = "blue"):
    if color.lower() == "blue":
        tup = (0.0, 0.0, 0.8)
    elif color.lower() == "red":
        tup = (0.8, 0.0, 0.0)
    else:
        tup = (0.0, 0.8, 0.0)
    for ic in range(r_coils.shape[0]):
        for n in range(r_coils.shape[2]):
            for b in range(r_coils.shape[3]):
                p = mlab.plot3d(r_coils[ic,:,n,b,0], r_coils[ic,:,n,b,1], r_coils[ic,:,n,b,2], tube_radius = 0.004, line_width = 0.01, color = tup)
    return p
def draw_surface(surface):
    r = surface.get_r()
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=(0.8,0.0,0.0))
    return p

mlab.clf()
draw_surface(Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, 32, 1.0))
draw_coils(r_fil)

In [ ]:
def compute_f_B(NZ, NT, NS):
    surface = Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", NZ, NT, 1.0)
    w_L = 0.1
    w_B = 1e3
    w_args = (w_B, w_L)
    coil_data_fil, params_fil = CoilSet.get_initial_data(surface, input_file="filament.hdf5")
    NC, _, NF, NFR, ln, lb, NNR, NBR, rc, NR = coil_data_fil
    coil_data_fil = NC, NS, NF, NFR, ln, lb, NNR, NBR, rc, NR
    _, _, r_fil, _, l_fil = CoilSet.get_outputs(coil_data_fil, False, params_fil)
    r_centroid_fil = CoilSet.get_r_centroid(coil_data_fil, False, params_fil)
    surface_data = (surface.get_r_central(), surface.get_nn(), surface.get_sg())
    coil_output_func_fil = partial(CoilSet.get_outputs, coil_data_fil, False)
    loss_fil = default_loss(surface_data, coil_output_func_fil, w_args, params_fil)
    f_B = loss_fil - w_L * l_fil
    return f_B

In [ ]:
NT = 32
NS = 64
NZs = [16, 32, 64, 96, 128, 192, 256, 320, 384, 448, 512]
f_Bs_NZ = []
for NZ in NZs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NZ.append(f_B)

In [ ]:
plt.plot(NZs, f_Bs_NZ)
plt.ylim([0.019,0.027])
plt.title("Quadratic flux vs num_zeta on surface")
plt.show()

In [ ]:
NT = 32
NSs = [16, 32, 64, 96, 128, 192, 256, 320, 384, 448, 512]
NZ = 128
f_Bs_NS = []
for NS in NSs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NS.append(f_B)

In [ ]:
plt.plot(NSs, f_Bs_NS)
plt.ylim([0.021,0.024])
plt.title("Quadratic flux vs num_segments in coils")
plt.show()

In [ ]:
NTs = [16, 32, 64, 96, 128, 192, 256, 320, 384, 448, 512]
NS = 64
NZ = 128
f_Bs_NT = []
for NT in NTs:
    f_B = compute_f_B(NZ, NT, NS)
    f_Bs_NT.append(f_B)

In [ ]:
plt.plot(NSs, f_Bs_NT)
plt.ylim([0.0212,0.0213])
plt.title("Quadratic flux vs num_theta on surface")
plt.show()

In [ ]:
def draw_surface(surface,color=(0.8,0.0,0.0)):
    r = surface.get_r()
    x = r[:,:,0]
    y = r[:,:,1]
    z = r[:,:,2]
    p = mlab.mesh(x,y,z,color=color)
    return p
mlab.clf()
draw_surface(Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 8, NT, 1.0))
draw_surface(Surface("../../focusadd/initFiles/axes/ellipticalAxis4Rotate.txt", 128, NT, 1.0), color=(0.0,0.8,0.0))